In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.colab import files

In [2]:
# Google Colab: Service Account Authentication + Fetch Events from May 6, 2025 Onward

# 1) (Optional) Upload your service-account JSON
from google.colab import files
uploaded = files.upload()  # Select your service-account JSON if not already in workspace

# 2) Authenticate with the service account
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timezone, timedelta

# Determine the credentials filename
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0] if uploaded else 'credentials.json'
SCOPES = ['https://www.googleapis.com/auth/calendar.events']

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=SCOPES
)
service = build('calendar', 'v3', credentials=creds)

# 3) Specify your shared calendar ID
# Replace the placeholder below with the exact Calendar ID you copied from the Google Calendar UI
MY_CAL_ID = 'yus404@gmail.com'

# 4) Set the starting time (May 6, 2025 at 00:00 CDT)
chicago_tz = timezone(timedelta(hours=-5))
time_min = datetime(2025, 5, 6, 0, 0, 0, tzinfo=chicago_tz).isoformat()

# 5) Fetch and print the next 5 events from May 6 onwards
events = service.events().list(
    calendarId=MY_CAL_ID,
    timeMin=time_min,
    maxResults=5,
    singleEvents=True,
    orderBy='startTime'
).execute().get('items', [])

print(f"Next 5 events on or after {time_min}:")
for ev in events:
    dt = ev['start'].get('dateTime', ev['start'].get('date'))
    print(f"{dt} — {ev.get('summary', '(no title)')}")


Saving service_account.json to service_account.json
Next 5 events on or after 2025-05-06T00:00:00-05:00:
2025-05-08T20:30:00-05:00 — Doctors Appointment w/ Dr. Alam (Shoulder)
2025-05-09T11:30:00-05:00 — Doctor Appointment W/Dr. Richardson (Knee)
2025-05-10T13:30:00-05:00 — Follow Up Doctors Appointment w/ Dr. Mitchell
2025-05-15T14:00:00-05:00 — Doctor Follow-Up
2025-05-15T14:00:00-05:00 — Doctor Follow-Up


In [3]:
from datetime import datetime, timedelta, timezone

# 1) Create a test event on May 9, 2025 at 10 AM Chicago time
test_event = {
    'summary': 'API Test Appointment',
    'start': {'dateTime': '2025-05-09T10:00:00-05:00'},
    'end':   {'dateTime': '2025-05-09T11:00:00-05:00'},
}

created = service.events().insert(
    calendarId=MY_CAL_ID,
    body=test_event
).execute()
print("Inserted:", created['summary'], created['start']['dateTime'])

# 2) Now re-fetch the next 5 events from May 6 onward
events = service.events().list(
    calendarId=MY_CAL_ID,
    timeMin=time_min,
    maxResults=5,
    singleEvents=True,
    orderBy='startTime'
).execute().get('items', [])

print("\nAfter insertion, events from May 6:")
for ev in events:
    dt = ev['start'].get('dateTime', ev['start'].get('date'))
    print(f"{dt} — {ev.get('summary','(no title)')}")


Inserted: API Test Appointment 2025-05-09T10:00:00-05:00

After insertion, events from May 6:
2025-05-08T20:30:00-05:00 — Doctors Appointment w/ Dr. Alam (Shoulder)
2025-05-09T10:00:00-05:00 — API Test Appointment
2025-05-09T11:30:00-05:00 — Doctor Appointment W/Dr. Richardson (Knee)
2025-05-10T13:30:00-05:00 — Follow Up Doctors Appointment w/ Dr. Mitchell
2025-05-15T14:00:00-05:00 — Doctor Follow-Up


In [4]:
from datetime import datetime, timezone, timedelta
def book_appointment(summary: str,
                     start_iso: str,
                     end_iso: str,
                     calendar_id: str = MY_CAL_ID) -> dict:
    """
    Creates a calendar event.

    • summary:        title of the event
    • start_iso:      RFC3339 timestamp, e.g. '2025-05-15T14:00:00-05:00'
    • end_iso:        RFC3339 timestamp, one hour later, e.g. '2025-05-15T15:00:00-05:00'
    • calendar_id:    your shared calendar’s ID
    """
    event = {
        'summary': summary,
        'start': {'dateTime': start_iso},
        'end':   {'dateTime': end_iso},
    }
    created = service.events().insert(
        calendarId=calendar_id,
        body=event
    ).execute()
    return created


# booking appt for May 15, 2025 from 2 PM to 3 PM Chicago time
start_time = '2025-05-15T14:00:00-05:00'
end_time   = '2025-05-15T15:00:00-05:00'

new_event = book_appointment(
    summary   = 'Doctor Follow-Up',
    start_iso = start_time,
    end_iso   = end_time
)

print("Booked event:")
print(" • Title:      ", new_event['summary'])
print(" • Start time: ", new_event['start']['dateTime'])
print(" • End time:   ", new_event['end']['dateTime'])
print(" • Event link: ", new_event.get('htmlLink'))


Booked event:
 • Title:       Doctor Follow-Up
 • Start time:  2025-05-15T14:00:00-05:00
 • End time:    2025-05-15T15:00:00-05:00
 • Event link:  https://www.google.com/calendar/event?eid=aDNhNDEwYXFpcm9kaDcxbDJoNGVoNXR0MnMgeXVzNDA0QG0
